In [1]:
!export CUDA_VISIBLE_DEVICES=1

In [2]:
config = """
Global:

Train:
  dataset:
    transforms:
      - DecodeImage: # load image
          img_mode: RGB
          channel_first: False
      - VQATokenLabelEncode: # Class handling label
          contains_re: False
          algorithm: Bert
          class_path: train_data/XFUND/class_list_xfun.txt
          use_textline_bbox_info: &use_textline_bbox_info True
          order_method: "tb-yx" # one of [None, "tb-yx"]
      - VQATokenPad:
          max_seq_len: &max_seq_len 512
          return_attention_mask: True
      - VQASerTokenChunk:
          max_seq_len: 512
      - Resize:
          size: [224,224]
      - NormalizeImage:
          scale: 1
          mean: [ 123.675, 116.28, 103.53 ]
          std: [ 58.395, 57.12, 57.375 ]
          order: 'hwc'
      - ToCHWImage:
      - KeepKeys:
          keep_keys: [ 'input_ids', 'bbox', 'attention_mask', 'token_type_ids', 'image', 'labels'] # dataloader will return list in this order
"""

import yaml

config = yaml.load(config, Loader=yaml.FullLoader)
dataset_config = config['Train']['dataset']
global_config = config['Global']

产生 batch 数据

In [3]:
import json
import os
from ppocr.data.imaug import create_operators, transform
import paddle

data_dir = "train_data/XFUND/zh_train/image"
label_file = "train_data/XFUND/zh_train/train.json"
with open(label_file, 'r') as f:
    lines = f.readlines()

line = lines[1].strip()
file_name, label = line.split('\t')
img_path = os.path.join(data_dir, file_name)

data = {'img_path': img_path, 'label': label}
with open(data['img_path'], 'rb') as f:
    img = f.read()
    data['image'] = img

ops = create_operators(dataset_config['transforms'], global_config)

outs = transform(data, ops)
outs = [paddle.to_tensor(out, place='cpu') for out in outs]
batch = [paddle.unsqueeze(out, axis=0) for out in outs]

[2022-12-16 08:33:32,661] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'bert-base-chinese'.
[2022-12-16 08:33:32,664] [    INFO] - Already cached /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt
[2022-12-16 08:33:32,676] [    INFO] - tokenizer config file saved in /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2022-12-16 08:33:32,677] [    INFO] - Special tokens file saved in /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


In [4]:
print(f"type of outs: {type(batch)}")
print(f"len of outs: {len(batch)}")
# keep_keys: [ 'input_ids', 'bbox', 'attention_mask',
# 'token_type_ids', 'image', 'labels']
print(f"shape of input_ids: {batch[0].shape}")
print(f"shape of bbox: {batch[1].shape}")
print(f"shape of attention_mask: {batch[2].shape}")
print(f"shape of token_type_ids: {batch[3].shape}")
print(f"shape of image: {batch[4].shape}")
print(f"shape of labels: {batch[5].shape}")


type of outs: <class 'list'>
len of outs: 6
shape of input_ids: [1, 512]
shape of bbox: [1, 512, 4]
shape of attention_mask: [1, 512]
shape of token_type_ids: [1, 512]
shape of image: [1, 3, 224, 224]
shape of labels: [1, 512]


tokenizer 使用实例

In [5]:
from paddlenlp.transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("layoutxlm-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
input_ids = batch[0][0].numpy().tolist()
print(tokenizer.decode(input_ids))

[2022-12-16 08:33:36,554] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'bert-base-chinese'.
[2022-12-16 08:33:36,557] [    INFO] - Already cached /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt
[2022-12-16 08:33:36,590] [    INFO] - tokenizer config file saved in /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2022-12-16 08:33:36,592] [    INFO] - Special tokens file saved in /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


中 国 人 体 器 官 捐 献 编 号 : [UNK] [UNK] 中 国 人 体 器 官 捐 献 志 愿 登 记 表 在 志 愿 登 记 前, 请 仔 细 阅 读 以 下 内 容 : 1. 人 体 器 官 捐 献 遵 循 [UNK] 自 愿 、 无 偿 [UNK] 的 原 则 。 2. 捐 献 发 生 在 逝 世 之 后, 不 会 影 响 对 您 的 抢 救 和 治 疗 。 3. 最 终 能 否 实 现 捐 献, 需 经 医 学 评 估 并 尊 重 亲 属 的 意 见 。 4. 请 将 捐 献 意 愿 告 知 家 人, 获 得 家 人 的 理 解 和 支 持 。 5. 如 果 捐 献 意 愿 发 生 改 变, 可 以 随 时 变 更 或 撤 销 。 我 具 备 完 全 民 事 行 为 能 力, 已 阅 读 并 知 悉 上 述 内 容, 自 愿 做 如 下 登 记 : 本 人 基 本 信 息 : 姓 名 : ( 身 份 证 / 护 照 ) 号 码 : 210422198210034379 任 星 二 联 系 电 话 : 18655212648 电 子 邮 箱 : 36214515 @ qq. com 登 记 地 : 辽 宁 省 （ 区 / 市 ） 抚 顺 市 （ 州 ） 县 （ 市 / 区 ） 我 志 愿 捐 献 : 器 官 [UNK] ( 眼 角 膜 □ 其 他 组 织 □ 遗 体 □ ) [UNK] 同 意 上 述 所 捐 用 于 临 床 医 疗 、 医 学 教 学 和 科 学 研 究 。 志 愿 登 记 者 签 名 : 任 星 二 2020 年 3 月 15 日 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

模型

In [6]:
from ppocr.modeling.architectures import build_model
from ppocr.modeling.backbones import build_backbone

config = {
    'model_type': 'kie',
    'Transform': None,
    'Backbone': {
        'name': 'BertForSer',
        'pretrained': True,
        'checkpoints': None,
        'mode': 'base',
        'num_classes': 7
    }
}

model = build_model(config)

[2022-12-16 08:33:39,429] [    INFO] - Already cached /mnt/sda/kaihong/.paddlenlp/models/bert-base-chinese/bert-base-chinese.pdparams
W1216 08:33:39.433658 497045 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 11.8, Runtime API Version: 11.7
W1216 08:33:39.446753 497045 gpu_resources.cc:91] device: 0, cuDNN Version: 8.5.
[2022-12-16 08:33:45,093] [    INFO] - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']


In [7]:
preds = model(batch)

print(f"type of preds: {type(preds)}")
print(f"len of preds: {len(preds)}")
print(f"type of preds: {type(preds)}")
print(f"shape of preds['backbone_out']: {preds['backbone_out'].shape}")

type of preds: <class 'dict'>
len of preds: 1
type of preds: <class 'dict'>
shape of preds['backbone_out']: [1, 512, 7]


后处理

In [8]:
from ppocr.postprocess import build_post_process

config = {
    "name": "VQASerTokenLayoutLMPostProcess",
    "class_path": "train_data/XFUND/class_list_xfun.txt"
}

batch_numpy = [out.numpy() for out in batch]
post_process_class = build_post_process(config, global_config)
post_result = post_process_class(preds['backbone_out'], batch_numpy)
print(f"len of post_result: {len(post_result)}")

len of post_result: 2


指标

In [9]:
from ppocr.metrics import build_metric

config = {
    "name": "VQASerTokenMetric",
    "main_indicator": "hmean",
}

eval_class = build_metric(config)
eval_class(post_result, batch)
metric = eval_class.get_metric()
print(metric)

{'precision': 0.0, 'recall': 0.0, 'hmean': 0.0}
